In [2]:
import pandas as pd
import re
import pickle
import datetime
import json
import numpy as np
import wikipedia as wiki
import tmdbsimple as tmdb

tmdb.API_KEY = 'ecb4864639691c0b1573d1cdaf39a336'
path_data= 'C:/root/Universita/project_dmdv/data/'


#some useful functions

#funzione per la sostituzione sulla base di una regex, la funzione nativa va in errore se il valore è null
def regex_replace(regex, sub, string):
    if pd.isnull(string):
        return None
    else:
        return re.sub(regex, sub, string)

#matching sulla base di una regex, la f nativa va in errore se il valore è mancante
def regex_match(regex, string, reverse = False):
    if pd.isnull(string):
        return None
    else:
        return re.match(regex, string)
    
#is_winner
def get_winner(string):
    if string:
        return 'Winner'
    else:
        return 'Nominee'
    
def replace_na_string(string):
    str_out = string
    if pd.notnull(string):
        if string.replace(' ', '') == '':
            str_out = None
    return str_out

def substring(string, start, end):
    str_out = None
    if pd.notnull(string):
        str_out = string[start:end]
    return str_out

In [2]:
#funzione per ottenere una lista di potenziali id di tmdb a partire dal titolo. Probabilmente si poteva gestire con ricorsivita e senza tutti gli if
#ma sono solo quattro, è inutile perderci tempo.
def get_id_movie(string):
    title_id = []
    res = tmdb.Search().movie(query =   string)
    if res['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res['results']]
    
    #elimino cio che sta nelle parentesi
    res0 = tmdb.Search().movie(query = regex_replace(r'\(.*\)','', string).strip())
    if res0['total_results'] > 0:
        title_id = title_id +  [i['id'] for i in res0['results']]
    #elimino cio che sta fuori dalle parentesi
    res1 = tmdb.Search().movie(query = regex_replace(r'\)', '',regex_replace(r'\)*[^()]*\(','', string)).strip())
    if res1['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res1['results']]
    
    #si prova con wikipedia
    wiki_list = wiki.search(string.lower() + ' film')
    if len(wiki_list) > 0:
        #elimino cio che sta nelle parentesi
        wiki_word = regex_replace(r'\(.*\)','', wiki_list[0]).strip()
        res = tmdb.Search().movie(query =   wiki_word)
        if res['total_results'] > 0:
            title_id = title_id + [i['id'] for i in res['results']]
    
    if len(title_id) == 0:
        title_id = None
    return title_id

In [3]:
#funzione per ottenere una lista di potenziali id di tmdb a partire dal titolo. Probabilmente si poteva gestire con ricorsivita e senza tutti gli if
#ma sono solo quattro, è inutile perderci tempo.
def get_id_tv(string):
    title_id = []
    res = tmdb.Search().tv(query =   string)
    if res['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res['results']]
    
    #elimino cio che sta nelle parentesi
    res0 = tmdb.Search().tv(query = regex_replace(r'\(.*\)','', string).strip())
    if res0['total_results'] > 0:
        title_id = title_id +  [i['id'] for i in res0['results']]
    #elimino cio che sta fuori dalle parentesi
    res1 = tmdb.Search().tv(query = regex_replace(r'\)', '',regex_replace(r'\)*[^()]*\(','', string)).strip())
    if res1['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res1['results']]
    
    #si prova con wikipedia
    wiki_list = wiki.search(string.lower() + ' tv')
    if len(wiki_list) > 0:
        #elimino cio che sta nelle parentesi
        wiki_word = regex_replace(r'\(.*\)','', wiki_list[0]).strip()
        res = tmdb.Search().tv(query =   wiki_word)
        if res['total_results'] > 0:
            title_id = title_id + [i['id'] for i in res['results']]
    
    if len(title_id) == 0:
        title_id = None
    return title_id

In [4]:
#funzione per ottenere una lista di potenziali id di tmdb a partire dal titolo. Probabilmente si poteva gestire con ricorsivita e senza tutti gli if
#ma sono solo quattro, è inutile perderci tempo.
def get_id_person(string):
    title_id = []
    res = tmdb.Search().person(query =   string)
    if res['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res['results']]
    
    #elimino cio che sta nelle parentesi
    res0 = tmdb.Search().person(query = regex_replace(r'\(.*\)','', string).strip())
    if res0['total_results'] > 0:
        title_id = title_id +  [i['id'] for i in res0['results']]
    #elimino cio che sta fuori dalle parentesi
    res1 = tmdb.Search().person(query = regex_replace(r'\)', '',regex_replace(r'\)*[^()]*\(','', string)).strip())
    if res1['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res1['results']]
    
    #si prova con wikipedia
    wiki_list = wiki.search(string.lower())
    if len(wiki_list) > 0:
        #elimino cio che sta nelle parentesi
        wiki_word = regex_replace(r'\(.*\)','', wiki_list[0]).strip()
        res = tmdb.Search().person(query =   wiki_word)
        if res['total_results'] > 0:
            title_id = title_id + [i['id'] for i in res['results']]
    
    if len(title_id) == 0:
        title_id = None
    return title_id

### Movies

In [5]:
data_award = pd.read_pickle(path_data + 'awards/movies_awards.data')
data_award.head()

,award,year,program,person,is_winner,award_type,title_upper,award_id
0,ACTOR,1927.0,The Noose,[Richard Barthelmess],Nominee,oscar award,THE NOOSE,aw-1
1,ACTOR,1927.0,The Last Command,[Emil Jannings],Winner,oscar award,THE LAST COMMAND,aw-2
2,ACTRESS,1927.0,A Ship Comes In,[Louise Dresser],Nominee,oscar award,A SHIP COMES IN,aw-3
3,ACTRESS,1927.0,7th Heaven,[Janet Gaynor],Winner,oscar award,7TH HEAVEN,aw-4
4,ACTRESS,1927.0,Sadie Thompson,[Gloria Swanson],Nominee,oscar award,SADIE THOMPSON,aw-5


In [ ]:
for i in range( data_award.shape[0]):
    award = data_award.iloc[i]['award_id']
    current_title = data_award.iloc[i]['title_upper']
    try:
        id_list = get_id_movie(current_title)
    except:
        id_list = []
    if i == 0:
        pd.DataFrame({'award_id' : award, 'tmdb_title_id' : [id_list]}).to_csv(path_data + 'custom/movie_id_api.csv',
                                                                               mode = 'w', index = False, header = True)
    else:
        pd.DataFrame({'award_id' : award, 'tmdb_title_id' : [id_list]}).to_csv(path_data + 'custom/movie_id_api.csv',
                                                                               mode = 'a', index = False, header = False)

### TV Series

In [12]:
data_award = pd.read_pickle(path_data + 'awards/tv_series_awards.data')
data_award.head()

,award,year,program,person,is_winner,type,award_type,title_upper,award_id
0,Comedy - Comedy Series,2019.0,Barry,"[Alec Berg, Bill Hader, Aida Rodgers, Emily He...",Nominee,T,emmy award,BARRY,aw-14028
1,Comedy - Comedy Series,2019.0,Fleabag,"[Phoebe Waller-Bridge, Harry Bradbeer, Lydia H...",Winner,T,emmy award,FLEABAG,aw-14029
2,Comedy - Comedy Series,2019.0,Russian Doll,"[Natasha Lyonne, Leslye Headland, Amy Poehler,...",Nominee,T,emmy award,RUSSIAN DOLL,aw-14030
3,Comedy - Comedy Series,2019.0,Schitt's Creek,"[Eugene Levy, Daniel Levy, David West Read, An...",Nominee,T,emmy award,SCHITT'S CREEK,aw-14031
4,Comedy - Comedy Series,2019.0,The Good Place,"[Michael Schur, David Miner, Morgan Sackett, D...",Nominee,T,emmy award,THE GOOD PLACE,aw-14032


In [ ]:
for i in range(data_award.shape[0]):
    award = data_award.iloc[i]['award_id']
    current_title = data_award.iloc[i]['title_upper']
    try:
        id_list_tv = get_id_tv(current_title)
    except:
        id_list_tv = []
    if data_award.iloc[i]['type'] == 'TM':
        try:
            id_list_movie = get_id_movie(current_title)
        except:
            id_list_movie = []
    if i == 0:
        pd.DataFrame({'award_id' : award, 'tmdb_tv_id' : [id_list_tv], 'tmdb_movie_id' : [id_list_movie]}).to_csv('tvs_id_api.csv',
                                                                               mode = 'w', index = False, header = True)
    else:
        pd.DataFrame({'award_id' : award, 'tmdb_tv_id' : [id_list_tv], 'tmdb_movie_id' : [id_list_movie]}).to_csv('tvs_id_api.csv',
                                                                               mode = 'a', index = False, header = False)

### People

In [4]:
data_award_movies = pd.read_pickle(path_data + 'awards/movies_awards.data')
data_award_tvs = pd.read_pickle(path_data + 'awards/tv_series_awards.data')
data_award.head()

,award,year,program,person,is_winner,award_type,title_upper,award_id
0,ACTOR,1927.0,The Noose,[Richard Barthelmess],Nominee,oscar award,THE NOOSE,aw-1
1,ACTOR,1927.0,The Last Command,[Emil Jannings],Winner,oscar award,THE LAST COMMAND,aw-2
2,ACTRESS,1927.0,A Ship Comes In,[Louise Dresser],Nominee,oscar award,A SHIP COMES IN,aw-3
3,ACTRESS,1927.0,7th Heaven,[Janet Gaynor],Winner,oscar award,7TH HEAVEN,aw-4
4,ACTRESS,1927.0,Sadie Thompson,[Gloria Swanson],Nominee,oscar award,SADIE THOMPSON,aw-5


In [7]:
data_award = pd.concat([data_award_movies,data_award_tvs]).explode('person')

In [9]:
data_award['id'] = [i+1 for i in range(data_award.shape[0])]

In [4]:
def get_id_person(string):
    title_id = []
    res = tmdb.Search().person(query =   string)
    if res['total_results'] > 0:
        title_id = title_id + [i['id'] for i in res['results']]
    
    #si prova con wikipedia
    wiki_list = wiki.search(string.lower())
    if len(wiki_list) > 0:
        #elimino cio che sta nelle parentesi
        wiki_word = wiki_list[0]
        res = tmdb.Search().person(query =   wiki_word)
        if res['total_results'] > 0:
            title_id = title_id + [i['id'] for i in res['results']]
    
    if len(title_id) == 0:
        title_id = None
    return title_id

In [5]:
for i in range(data_award.shape[0]):
    award = data_award.iloc[i]['award_id']
    if pd.notnull(data_award.iloc[i]['person']):
        current_person = data_award.iloc[i]['person'].upper()
        try:
            id_list = get_id_person(current_person)
        except:
            id_list = []
    else:
        current_person = data_award.iloc[i]['person']
        id_list = []
    if i == 0:
        pd.DataFrame({'award_id' : award, 'person' : current_person, 'tmdb_person_id' : [id_list]}).to_csv(path_data + 'custom/people_id_api.csv',
                                                                               mode = 'w', index = False, header = True)
    else:
        pd.DataFrame({'award_id' : award, 'person' : current_person, 'tmdb_person_id' : [id_list]}).to_csv(path_data + 'custom/people_id_api.csv',
                                                                               mode = 'a', index = False, header = False)